In [40]:
import os
import pandas as pd

In [91]:
def read_raw_spectra(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list
    
    wns = []
    sp1 = []
    sp2 = []
    sp3 = []
    sp4 = []
    times = lines[0].split()
    
    temp1 = [float(b) for b in (lines[1].split())[6:]]
    temp2 = [float(b) for b in (lines[2].split())[6:]]
    temp3 = [float(b) for b in (lines[3].split())[6:]]
    temp4 = [float(b) for b in (lines[4].split())[6:]]
    
    for i in lines[5:]:
        line = i.split()
        final = [float(a) for a in line]
        
        wns.append(final[0])
        sp1.append(final[1])
        sp2.append(final[2])
        sp3.append(final[3])
        sp4.append(final[4])

    return wns, sp1, sp2, sp3, sp4, times, temp1, temp2, temp3, temp4

In [94]:
wns, spec1, spec2, spec3, spec4, times, temp1, temp2, temp3, temp4 = read_raw_spectra('//icnas4.cc.ic.ac.uk/rs5115/Msci Project/raw_spectra', 'IC_raw_spectra_20.txt')

In [95]:
spec1

[0.034026496,
 0.034863204,
 0.035393342,
 0.036203723,
 0.036074664,
 0.035186645,
 0.035279904,
 0.036175817,
 0.037464689,
 0.039228018,
 0.040521845,
 0.04141736,
 0.042387325,
 0.04323132,
 0.044810545,
 0.047013901,
 0.047763832,
 0.047532722,
 0.048674684,
 0.050797332,
 0.052696064,
 0.054226305,
 0.055120301,
 0.055859279,
 0.057449862,
 0.059832193,
 0.061592896,
 0.062395651,
 0.063953362,
 0.06600932,
 0.067903623,
 0.070883282,
 0.073748946,
 0.075745434,
 0.078214966,
 0.080151729,
 0.081223458,
 0.083083093,
 0.086181045,
 0.089492217,
 0.091943912,
 0.093795821,
 0.095651992,
 0.097793072,
 0.100579359,
 0.103764459,
 0.106980063,
 0.11065539,
 0.114305511,
 0.116530955,
 0.117885455,
 0.119505003,
 0.121585779,
 0.125001773,
 0.129577473,
 0.133552924,
 0.136970043,
 0.140288219,
 0.142561644,
 0.144651502,
 0.148011684,
 0.151803702,
 0.155818149,
 0.160584331,
 0.164556488,
 0.166800603,
 0.168427244,
 0.17058903,
 0.174458221,
 0.179654643,
 0.184841648,
 0.18992276